# 准备

## 库 导入

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
import warnings
import GEOparse
from typing import Tuple, List
from sklearn.metrics import  roc_curve, auc, precision_recall_curve, confusion_matrix, classification_report, accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
from multiprocessing import Pool, cpu_count
import ast


warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LinearRegression, RidgeClassifier, Lasso, LassoCV, LogisticRegression, LogisticRegressionCV, ElasticNet,SGDClassifier,RidgeClassifierCV
from sklearn.svm import SVC, LinearSVC, NuSVC

## 基础设置

In [ ]:
GSE_Train = "GSE162330" # 用于训练的数据集
Skip_Paint = True # 是否绘制不必要的绘图


In [ ]:
methods = [chi2, f_classif, mutual_info_classif]
models = [
    LinearRegression(), 
    RidgeClassifierCV(class_weight='balanced'),
    LogisticRegression(),
    Lasso(),
    LinearSVC(
        penalty='l1',
        loss='squared_hinge',
        dual=False,
        C=0.01,
        max_iter=10000,
        multi_class='ovr',
        fit_intercept=True,
        intercept_scaling=1,
        class_weight='balanced',
        verbose=0,
        random_state=0,
        tol=1e-4
    ),
    NuSVC(kernel='linear',
        max_iter=10000)
]
n_features = [i for i in range(1, 16)]

## 数据 导入

In [ ]:
GPL10558 = GEOparse.get_GEO(geo="GPL10558", destdir="./datasets", silent=True)

In [ ]:
gpl = GEOparse.get_GEO(geo='GPL10558', destdir="./datasets", silent=True)
# ./datasets/GPL10558.pkl
gsedata = pd.read_pickle("./datasets/GSE162330.pkl")

In [ ]:
gsedata.isnull().sum() 

In [ ]:
# drop the rows with missing values more than 50%
gsedata = gsedata.dropna(axis=1, thresh=0.5*len(gsedata.index))
gsedata = gsedata.dropna(axis=0, thresh=0.5*len(gsedata.columns))

gsedata.isnull().sum()

In [ ]:
gsedata['characteristics: sample class'].value_counts() # 看一下感染这个列的分布

## 函数 准备

#### 定义存储结果的DataFrame

In [ ]:
final_results = pd.DataFrame(columns=['method', 'model', 'n', 'acc', 'auc', 'f1', 'precision', 'recall', 'confusionmatrix','coef', 'coef_rounded', 'found_coef', 'multiple_times','intercept', 'gene_id', 'gene_symbol', 'gb_acc'])

#### 可接受的权重的范围筛选器

In [ ]:
def is_prime_factors_only(n, allowed_factors):
    for factor in allowed_factors:
        while n % factor == 0:
            n //= factor
    return n == 1

def get_allowed_numbers(max_n=25,allowed_factors=[2,3,5]):
    result = []
    for i in range(1, max_n + 1):
        if is_prime_factors_only(i, allowed_factors):
            result.append(i)
    return result

def is_all_in_list(list,n=25,allowed_factors=[2,3,5]):
    for i in list:
        if i not in get_allowed_numbers(n,allowed_factors):
            return False
    return list

### 画图

分数分布图

In [ ]:
def plot_score_distribution(ax, y_score, final_target):                                 # 这个函数负责处理分数分布图这一个小图
    ax.scatter(range(len(y_score)), y_score, c=final_target, cmap='bwr', alpha=0.5)     # 画散点图，x轴是样本序号，y轴是分数，颜色是感染情况，颜色映射是蓝白红，透明度是0.5
                                                                                        # x是样本序号：分开画，免得都黏在一起了，透明度的设置是一样的原因。
    ax.set_title('分数分布图')
    ax.set_xlabel('样本序号')
    ax.set_ylabel('分数')
    plt.rcParams['font.sans-serif']=['SimHei']                                          # 中文乱码
    plt.rcParams['axes.unicode_minus']=False                                            # 负号乱码
    labels = ['细菌感染', '病毒感染']                                                   # 图例标签，这里是感染情况，以后如果换了的话得改。 #todo 将这个改为传入参数
    ax.legend(labels, loc='upper right')                                                # 图例
    ax.plot([0, len(y_score)], [0.5, 0.5], color='black', lw=1, linestyle='--')         # 以0.5为界，画一条虚线，表示分数大于0.5的是细菌感染，小于0.5的是病毒感染，这个以后也得改
                                                                                        # 因为不同的模型（如linear regression 和 ridge regression）的判断标准不一样，所以这个界限也不一样。
                                                                                        # 现在的做法是直接在y_score 上做手脚，还是……蛮不优雅的，但是先这样吧。
                                                                                        # todo 将这个改为传入参数

ROC曲线

In [ ]:
def plot_roc_curve(ax, y_score, final_target):
    from sklearn.metrics import auc
    fpr, tpr, thresholds = roc_curve(final_target, y_score)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax.set_xlabel('False Positive Rate 假阳性率')
    ax.set_ylabel('True Positive Rate 真阳性率')
    ax.set_title('ROC 曲线（受试者工作特征曲线）')
    ax.legend(loc="lower right")

PR曲线

In [ ]:
def plot_pr_curve(ax, y_score, final_target):
    precision, recall, thresholds = precision_recall_curve(final_target, y_score)
    ax.plot(recall, precision, color='darkorange', lw=2, label='PR curve')
    ax.set_xlabel('Recall 召回率')
    ax.set_ylabel('Precision 准确率')
    ax.set_title('PR 曲线（准确率-召回率曲线）')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.legend(loc="lower left")

混淆矩阵

In [ ]:
def plot_confusion_matrix(ax, y_score, final_target):
    y_pred = np.where(y_score > 0.5, 1, 0)
    cnf_matrix = confusion_matrix(final_target, y_pred)
    ax.imshow(cnf_matrix, interpolation='nearest',cmap=plt.cm.Blues) # type: ignore
    ax.set_title('Confusion Matrix 混淆矩阵')
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['病毒感染', '细菌感染'], rotation=45)
    plt.yticks(tick_marks, ['病毒感染', '细菌感染'])
    thresh = cnf_matrix.max() / 2.
    for i in range(cnf_matrix.shape[0]):
        for j in range(cnf_matrix.shape[1]):
            plt.text(j, i, cnf_matrix[i, j],horizontalalignment="center",color="white" if cnf_matrix[i, j] > thresh else "black",fontsize=20)
    plt.tight_layout()
    plt.ylabel('True label 感染情况')
    plt.xlabel('Predicted label 预测情况')

分数分布图、ROC曲线、PR曲线和混淆矩阵

In [ ]:
def plot_all(score, target, title="分数分布图、ROC曲线、PR曲线和混淆矩阵", feature=None, weight=None):
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))                                                     # 规定画布大小为1200*1200px，分成2*2的4个小图
    fig.suptitle(title, fontsize=16)                                                                    # 设置标题
    plot_score_distribution(axs[0,0], score, target)                                                    # 第一个小图（左上角）是分数分布图 这里的axs[0,0]是指第一行第一列的小图，注意计数是从0开始的。
    plot_roc_curve(axs[0,1], score, target)                                                             # 第二个小图（右上角）是ROC曲线
    plot_pr_curve(axs[1,0], score, target)                                                              # 第三个小图（左下角）是PR曲线
    plot_confusion_matrix(axs[1,1], score, target)                                                      # 第四个小图（右下角）是混淆矩阵
    if feature or weight:                                                                               # 如果有feature或者weight的话，就在图的下方加上文字
        try:
            # add margin at bottom
            fig.subplots_adjust(bottom=0.12)                                                            # 调整图的下边距，这样文字就不会挡住图了
            # add feature and weight at the bottom
            fig.text(0.5, 0.04, '使用feature:'+str(feature), ha='center', va='bottom', fontsize=12)     # 在图的下方加上文字，这里的0.5,0.04是指文字的位置，ha='center'是指文字居中，va='bottom'是指文字在底部，fontsize=12是指文字大小
            fig.text(0.5, 0.02, '对应权重:'+str(weight), ha='center', va='bottom', fontsize=8)          # 同上
        except:
            pass                                                                                        # 如果出错了就算了
    return fig, axs

##### 给定 基因 权重 和数据，计算分数并画图（可能需要修改）

最简单

In [ ]:
def testfeature(feature,weight,data,gpl=gpl,title="分数分布图、ROC曲线、PR曲线和混淆矩阵"):
    ID = []
    for i in range(len(feature)):
        ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0]) # type: ignore
    ID = np.array(ID).reshape(-1)
    testdata = data[ID]
    n = len(testdata)
    y_score = np.zeros(n)
    y_pred = np.zeros(n)
    for i in range(len(testdata)):
        y_score [i] = np.dot(testdata.iloc[i],weight)
        y_pred [i] = np.where(y_score[i] > 0.5, 1, 0)
    try:
        target = data['target'].values
    except:
        target = gsedata['target']
    acc = accuracy_score(target, y_pred)
    print("Accuracy: ", acc)
    fig, axs = plot_all(y_score, target, title, feature=feature, weight=weight)

权重由放大得到

In [ ]:
def testfeaturewithtimes(feature,weight,data,times=1,gpl=gpl,title="分数分布图、ROC曲线、PR曲线和混淆矩阵"):
    ID = []
    for i in range(len(feature)):
        ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0]) # type: ignore
    ID = np.array(ID).reshape(-1)
    testdata = data[ID]
    n = len(testdata)
    y_score = np.zeros(n)
    y_pred = np.zeros(n)
    for i in range(len(testdata)):
        y_score [i] = np.dot(testdata.iloc[i],weight)
        # 结果除以times
        y_score [i] = y_score [i] / times
        y_pred [i] = np.where(y_score[i] > 0.5, 1, 0)
    try:
        target = data['target'].values
    except:
        target = gsedata['target']
    acc = accuracy_score(target, y_pred)
    print("Accuracy: ", acc)
    fig, axs = plot_all(y_score, target, title, feature=feature, weight=weight)

画出按照倍数比例画出各种基因在不同条件下的表达量

In [ ]:
def plot_boxplot(data_higher, gpl, gsedata, data_higher_times):
    fig, ax = plt.subplots(1,50, figsize=(50,15))
    for i in range(50):
        sns.boxplot(
            x='target', 
            y=data_higher.iloc[:,i],
            data=gsedata,
            ax=ax[i],
            )
        ax[i].set_title(gpl.table[gpl.table['ID'] == data_higher.columns[i]]['Gene Symbol'].values[0])

        # 最上面加上对应的倍数
        ax[i].text(0.1, 0.98,
                   str('%.2f' % data_higher_times[i]),
                   transform=ax[i].transAxes,
                   fontsize=16,
                   verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        # 显示优化
        ax[i].set_yscale('log') #对数坐标
        ax[i].set_ylim(10, 40000) # y轴的范围
        if i != 0: # 不显示x，y轴的标签， 不然太挤了没意义
            ax[i].set_xlabel('')
            ax[i].set_yticklabels([])
            ax[i].set_ylabel('')
            ax[i].set_yticks([])
        else:
            ax[i].set_xlabel('')
            ax[i].set_ylabel('表达量')
        ax[i].tick_params(axis='x', rotation=90) # 不然画不下
    plt.show()

### 筛选

按照n增加而计算准确率等

In [ ]:
def find_best_n(ax,X_train, y_train, X_test, y_test, method, model, max_features=20):
    train_acc,test_acc,train_roc,test_roc = [],[],[],[]
    for i in range(1, max_features):
        if method == 'mannual':
            reselect = X_train.iloc[:, :i]
            model.fit(reselect, y_train) # use the selected features to train the data
            y_pred = model.predict(X_test.iloc[:, :i])
            train_acc.append(model.score(reselect, y_train))
            test_acc.append(model.score(X_test.iloc[:, :i], y_test))
            train_roc.append(roc_auc_score(y_train, model.predict(reselect)))
            test_roc.append(roc_auc_score(y_test, model.predict(X_test.iloc[:, :i])))
        else:
            reselect = SelectKBest(score_func=method, k=i)
            reselect.fit(X_train, y_train)
            model.fit(X_train.iloc[:, reselect.get_support(indices=True)], y_train) # use the selected features to train the data
            y_pred = model.predict(X_test.iloc[:, reselect.get_support(indices=True)])
            train_acc.append(model.score(X_train.iloc[:, reselect.get_support(indices=True)], y_train))
            test_acc.append(model.score(X_test.iloc[:, reselect.get_support(indices=True)], y_test))
            train_roc.append(roc_auc_score(y_train, model.predict(X_train.iloc[:, reselect.get_support(indices=True)])))
            test_roc.append(roc_auc_score(y_test, model.predict(X_test.iloc[:, reselect.get_support(indices=True)])))
    # ax.plot(range(1, max_features), train_acc, label='train_acc')
    ax.plot(range(1, max_features), test_acc, label='test_acc')
    # ax.plot(range(1, max_features), train_roc, label='train_roc')
    ax.plot(range(1, max_features), test_roc, label='test_roc')
    if method == 'mannual':
        ax.set_title('使用'+str(model) + '按照表达比选择特征')
    else:
        ax.set_title('使用'+str(model) + '和' + method.__name__ + '选择特征')
    plt.rcParams['font.sans-serif'] = ['SimHei']
    ax.legend()
    # grid
    ax.grid()
    # mark the maximum value
    ax.scatter(test_acc.index(max(test_acc)) + 1, max(test_acc), marker='o', color='b')
    ax.text(test_acc.index(max(test_acc)) + 1, max(test_acc), str(test_acc.index(max(test_acc)) + 1) + ', ' + str('%.2f%%' % (max(test_acc) * 100)))
    ax.scatter(test_roc.index(max(test_roc)) + 1, max(test_roc), marker='o', color='r')
    ax.text(test_roc.index(max(test_roc)) + 1, max(test_roc), str(test_roc.index(max(test_roc)) + 1) + ', ' + str('%.2f%%' % (max(test_roc) * 100)))
    # y is 0.5 to 1
    ax.set_ylim(0.5, 1)

    if method != 'mannual':
        select = SelectKBest(score_func=method, k=test_acc.index(max(test_acc)) + 1).fit(X_train, y_train)
        X_train_new = select.transform(X_train)
        X_test_new = select.transform(X_test)
        model.fit(X_train_new, y_train)
        y_pred = model.predict(X_test_new)
        gene_id = X_train.columns[select.get_support(indices=True)]
    else:
        X_train_new = X_train.iloc[:, :test_acc.index(max(test_acc)) + 1]
        X_test_new = X_test.iloc[:, :test_acc.index(max(test_acc)) + 1]
        model.fit(X_train_new, y_train)
        y_pred = model.predict(X_test_new)
        gene_id = X_train_new.columns
    gene_symbol = [gpl.table[gpl.table['ID'] == i]['Gene Symbol'].values[0] for i in gene_id] # type: ignore
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的特征为：', gene_symbol)
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的特征权重为：', model.coef_)
    
    n = test_acc.index(max(test_acc)) + 1
    best_acc = max(test_acc)

    return best_acc, n, gene_symbol, model.coef_


筛选哪些基因比较有差异

In [ ]:
def get_higher_columns(data):
    data_mean = data.groupby('target').mean()
    higher_columns = []
    higher_times = []
    for i in range(data_mean.shape[1]):
        if data_mean.iloc[1,i] > 2*data_mean.iloc[0,i] or 2*data_mean.iloc[0,i] < data_mean.iloc[1,i]:
            higher_columns.append(data_mean.columns[i])
            higher_times.append(data_mean.iloc[1,i]/data_mean.iloc[0,i])
    return higher_columns, higher_times

排序有差异的基因

In [ ]:
def sort_higher_columns(data, higher_columns, higher_times):
    # sort the columns and times together
    data_higher_times, data_higher_columns = zip(*sorted(zip(higher_times, higher_columns), reverse=True))
    # convert data_higher_columns to list
    data_higher_columns = list(data_higher_columns)
    # add target column to the top of data_higher_columns
    data_higher_columns.append('target')
    # get the new data
    data_higher = data[data_higher_columns]
    return data_higher, data_higher_times

输出

In [ ]:
def print_best_acc(test_acc, n, gene_symbol, coef):
    best_acc = max(test_acc)
    best_i = 0
    i = 0
    print('最好的准确率为：', best_acc)
    for acc in test_acc:
        if acc == best_acc:
            print('='*60)
            print('对应的特征数为：', n[i],'方法为：', methods[i // len(models)].__name__,'特征为：', gene_symbol[i])
            print('模型为：', models[i % len(models)],'特征权重为：', coef[i])
            weights_avg = np.mean(coef[i])
            new_coefs = []
            for j in range(len(coef[i])):
                # new_coefs.append(10*(coef[i][j]/weights_avg)) 保留2位小数
                # new_coefs.append(round(10*(coef[i][j]/weights_avg), 2)) type numpy.ndarray doesn't define __round__ method
                new_coefs.append(np.round(10*(coef[i][j]/weights_avg), 2))
            print('比例大致为：', new_coefs)
        i += 1

不记得了

In [ ]:
def get_symbols(gene_symbols):
    gene_symbols = gene_symbols.reset_index(drop=True)
    symbol = []
    for i in range(len(gene_symbols)):
        for j in range(len(gene_symbols[i])):
            if gene_symbols[i][j] not in symbol:
                symbol.append(gene_symbols[i][j])
            else:
                pass
    return symbol


test all
输入筛选方法和训练模型等 输出结果

In [ ]:
# input: methods, models, n_features, X_train, X_test, y_train, y_test, gpl
# output: results


def test_all(methods, models, n_features, X_train, X_test, y_train, y_test, gpl):
    import time
    global final_results
    for method in methods:
        for n in n_features:
            reselect = SelectKBest(score_func=method, k=n)
            reselect.fit(X_train, y_train)
            for model in models:
                # log the time
                time_start = time.time()
                model.fit(reselect.transform(X_train), y_train)
                y_score = model.predict(reselect.transform(X_test))
                y_pred = np.array([1 if i > 0.5 else 0 for i in y_score], dtype=int)
                acc = accuracy_score(y_test, y_pred)
                auc = roc_auc_score(y_test, model.predict(reselect.transform(X_test)))
                f1 = f1_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                confusionmatrix = confusion_matrix(y_test, y_pred)
                coef = model.coef_

                weights_avg = np.mean(coef)
                min_times = 0.5 / np.min(np.abs(coef)) if np.min(coef) != 0 else 0.5 / np.mean(np.abs(coef))
                max_times = 25 / np.max(np.abs(coef))
                multipletimes = np.linspace(min_times, max_times, num=1000)
                found_coef = False
                coef_roundeds = []
                differences = []
                for multipletime in multipletimes:
                    coef_rounded = np.round(coef.ravel() * multipletime).tolist()
                    try:
                        coef_rounded = [int(i) for i in coef_rounded]
                        abs_coef_rounded = [abs(i) for i in coef_rounded]
                        if is_all_in_list(abs_coef_rounded):
                            coef_roundeds.append(coef_rounded)
                            difference = np.mean(np.abs(np.array(coef_rounded) - np.array(coef.ravel() * multipletime))) / np.mean(np.abs(np.array(coef_rounded)))
                            differences.append(difference)
                            found_coef = True
                        else:
                            pass
                    except:
                        pass
                if not found_coef:
                    coef_rounded = np.round(100 * coef.ravel() / weights_avg).tolist()
                    try:
                        coef_rounded = [int(i) for i in coef_rounded]
                    except:
                        pass
                    multiple_time = 100
                else:
                    best_index = differences.index(min(differences))
                    coef_rounded = coef_roundeds[best_index]
                    multiple_time = np.mean(np.abs(coef_rounded)) / np.mean(np.abs(coef.ravel()))
                    # is all > 0
                    # if not all(i > 0 for i in coef_rounded):
                    #     found_coef = False

                intercept = model.intercept_
                gene_id = X_train.columns[reselect.get_support(indices=True)]
                gene_id = np.array(gene_id).reshape(-1)
                gene_symbol = [gpl.table[gpl.table['ID'] == i]['Gene Symbol'].values[0] for i in gene_id]
                gb_acc = [gpl.table[gpl.table['ID'] == i]['GB_ACC'].values[0] for i in gene_id]
                final_results = final_results.append({
                    'method': method.__name__,
                    'model': model.__class__.__name__,
                    'n': n,
                    'acc': acc,
                    'auc': auc,
                    'f1': f1,
                    'precision': precision,
                    'recall': recall,
                    'confusionmatrix': confusionmatrix,
                    'coef': coef,
                    'coef_rounded': coef_rounded,
                    'found_coef': found_coef,
                    'multiple_times': multiple_time,
                    'intercept': intercept,
                    'gene_id': gene_id,
                    'gene_symbol': gene_symbol,
                    'gb_acc': gb_acc
                }, ignore_index=True)
                time_end = time.time()
                if time_end - time_start > 2:
                    print('SLOW!!!:method:', method.__name__, 'n:', n, 'model:', model.__class__.__name__,'time cost', time_end - time_start, 's')
    return results

plot_all_results 未使用

In [ ]:
# input: results, methods, models, gpl
# output: figure
def plot_all_results(results, methods, models, gpl):
    # Group the results by method and model
    groups = results.groupby(['method', 'model'])

    # Create a figure with multiple subplots
    fig, axs = plt.subplots(len(methods), len(models) + 1, sharex=True, sharey=True, figsize=(5*len(models), 3*(len(methods)+1)))

    # Iterate over the subplots and plot the acc and roc values
    for i, method in enumerate(methods):
        for j, model in enumerate(models):
            group = groups.get_group((method.__name__, model.__class__.__name__))
            axs[i, j].plot(group['n'], group['acc'], label='acc')
            axs[i, j].plot(group['n'], group['auc'], label='auc')
            axs[i, j].set_xlabel('n')
            axs[i, j].set_ylabel('score')
            axs[i, j].set_ylim([0.5, 1]) # Set y-axis limits
            axs[i, j].legend()
            # Find the maximum acc and roc values for this method and model
            max_acc_idx = group['acc'].idxmax()
            max_auc_idx = group['auc'].idxmax()
            max_acc = group.loc[max_acc_idx, 'acc']
            max_auc = group.loc[max_auc_idx, 'auc']
            max_n_acc = group.loc[max_acc_idx, 'n']
            max_n_auc = group.loc[max_auc_idx, 'n']

            # Add annotations with the maximum acc and roc values
            axs[i, j].annotate(f'Max Acc: {max_acc:.2f} (n={max_n_acc})',
                                xy=(max_n_acc, max_acc),
                                xytext=(max_n_acc, max_acc + 0.1),
                                ha='center',
                                arrowprops=dict(arrowstyle='->'))
            axs[i, j].annotate(f'Max Roc: {max_auc:.2f} (n={max_n_auc})',
                                xy=(max_n_auc, max_auc),
                                xytext=(max_n_auc, max_auc - 0.1),
                                ha='center',
                                arrowprops=dict(arrowstyle='->'))

            axs[i, j].set_title(f'{method.__name__} - {model.__class__.__name__}')
            # grid
            axs[i, j].grid()
        # Create a table of gene symbols for the first model in the method's group
        first_model = groups.get_group((method.__name__, models[0].__class__.__name__))
        # gene_symbols = first_model['gene_symbol'].reset_index(drop=True)
        # symbol = get_symbols(gene_symbols)
        gene_ids = first_model['gene_id'].reset_index(drop=True)
        id = get_symbols(gene_ids)
        # cell_text = np.array(symbol).reshape(-1, 1)
        cell_text = np.array([id]).T
        row_labels = [f'Gene {i}' for i in range(len(id))]
        col_labels = ['Gene Symbol', 'Gene ID']
        table = axs[i, len(models)].table(cellText=cell_text, rowLabels=row_labels, colLabels=col_labels, loc='center')
        axs[i, len(models)].axis('off')
        axs[i, len(models)].set_title(f'{method.__name__} - selected genes')

    # Iterate over the first column of subplots and set their y-axis labels to be the method names
    for i, method in enumerate(methods):
        axs[i, 0].set_ylabel(method.__name__)

    # Add a main title to the figure
    fig.suptitle('Accuracy and AOC by method and model')

    # Adjust the layout of the subplots
    fig.tight_layout()

    # Show the figure
    plt.show()

给定数据、选取基因和单向权重，计算分数

In [ ]:
def cal_score_with_feature(data, gpl, feature, weight, times, intercept,model):
    ID = []
    for i in range(len(feature)):
        ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0])
    ID = np.array(ID).reshape(-1)
    ID.astype('str')
    data = data[ID]
    data.columns = feature
    data_weight = data * weight
    data_weight_sum = data_weight.sum(axis=1)
    if model in ['LogisticRegression','LogisticRegressionCV','logisticregression','LR','lr']:
        data_weight_z = data_weight_sum / times + intercept
        data_weight_score = 1 / (1 + np.exp(-data_weight_z))
    elif model in ['LinearRegression','RidgeClassifier','Lasso','LassoCV','ElasticNet']:
        data_weight_score = data_weight_sum / times + intercept
    elif model in ['NuSVC','LinearSVC','SVC']:
        data_weight_score = data_weight_sum / times + intercept
    else:
        print('The model',model,'is not supported yet.')
        data_weight_score = data_weight_sum / times + intercept
    return data_weight_score

一次性完成基因筛选和模型训练
通过调控C，使用LogisticRegression or LinearSVC 一次性完成基因筛选和模型训练

In [ ]:
def evaluate_classifier(clf, method, model, X_train, y_train, X_test, y_test, params):
    global final_results

    acc_scores, auc_scores, f1_scores, precision_scores, recall_scores, n_features = [], [], [], [], [], []

    for param in params:
        clf.set_params(**param)
        clf.fit(X_train, y_train)
        nonzero_coef_indices = np.flatnonzero(clf.coef_)  # Indices where coef_ != 0
        n = len(nonzero_coef_indices)
        y_pred = clf.predict(X_test)
        y_pred = np.array([1 if i > 0.5 else 0 for i in y_pred], dtype=int)
        
        if 3 <= n <= 45:
            cm = confusion_matrix(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            acc = clf.score(X_test, y_test)

            coef = clf.coef_
            # only use non-zero coef
            coef = coef[coef != 0]
            weights_avg = np.mean(coef)
            min_times = 0.5 / np.min(np.abs(coef)) if np.min(coef) != 0 else 0.5 / np.mean(np.abs(coef))
            max_times = 25 / np.max(np.abs(coef))
            multipletimes = np.linspace(min_times, max_times, num=1000)
            found_coef = False
            coef_roundeds = []
            differences = []
            for multipletime in multipletimes:
                coef_rounded = np.round(coef.ravel() * multipletime).tolist()
                try:
                    coef_rounded = [int(i) for i in coef_rounded]
                    abs_coef_rounded = [abs(i) for i in coef_rounded]
                    if is_all_in_list(abs_coef_rounded):
                        coef_roundeds.append(coef_rounded)
                        difference = np.mean(np.abs(np.array(coef_rounded) - np.array(coef.ravel() * multipletime))) / np.mean(np.abs(np.array(coef_rounded)))
                        differences.append(difference)
                        found_coef = True
                        times = multipletime
                        break
                    else:
                        pass
                except:
                    pass
            if not found_coef:
                coef_rounded = np.round(100 * coef.ravel() / weights_avg).tolist()
                try:
                    coef_rounded = [int(i) for i in coef_rounded]
                except:
                    pass
                times = 100

            intercept = clf.intercept_

            nonzero_coef_indices = np.flatnonzero(clf.coef_) # get the indices of non-zero coef
            gene_id = list(X_train.columns[nonzero_coef_indices])
            gene_symbol = list(X_train.columns[nonzero_coef_indices])
            gb_acc = list(X_train.columns[nonzero_coef_indices])

            result = pd.DataFrame({
                'method': [method],
                'model': [model],
                'n': [n],
                'acc': [acc],
                'auc': [auc],
                'f1': [f1],
                'precision': [precision],
                'recall': [recall],
                'confusionmatrix': [cm],
                'coef': [coef.tolist()],
                'coef_rounded': [coef_rounded],
                'found_coef': [found_coef],
                'multiple_times': [times],
                'intercept': [intercept.item()],
                'gene_id': [gene_id],
                'gene_symbol': [gene_symbol],
                'gb_acc': [gb_acc]
            })

            final_results = pd.concat([final_results, result], ignore_index=True) 

            # For plotting
            n_features.append(n) 
            auc_scores.append(auc)
            f1_scores.append(f1)
            precision_scores.append(precision)
            recall_scores.append(recall)

    # Plotting inside the function
    plt.figure(figsize=(10,6))
    plt.plot(n_features, auc_scores, label='AUC')
    plt.plot(n_features, f1_scores, label='F1 score')
    plt.plot(n_features, precision_scores, label='Precision', linestyle='dashed')
    plt.plot(n_features, recall_scores, label='Recall', linestyle='dashed')
    plt.title('Performance metrics by number of features (' + method + ')')
    plt.xlabel('Number of features')
    plt.ylabel('Performance metrics')
    plt.legend()
    plt.grid()
    plt.show()

# 训练

### 只保留细菌和病毒

In [ ]:
target_map = {
    'virus': 1,
    'bacteria': 0,
    'viral': 1,
    'bacterial': 0,
    'S. aureus, MRSA': 0,
    'Influenza A': 1,
    'E. coli': 0,
    'S. pneumoniae': 0,
    'S. aureus, MSSA': 0,
    'None': -1,
    'Influenza B': 1,
    'non-infectious illness': -1,
    'v': 1,
    'b': 0,
    'c': -1
} 

In [ ]:
# 病毒与未感染的鉴别
train_data = gsedata.copy()
# prevent Object of type "Series[Unknown]" is not callable
train_data['target'] = train_data['characteristics: sample class'].map(target_map)
train_data = train_data[train_data['target'] != -1]

### 筛选出 细菌病毒感染中表达量大差异于等于2倍的基因

In [ ]:
higher_columns, higher_times = get_higher_columns(train_data)

根据倍率不同进行排序

In [ ]:
data_higher, data_higher_times = sort_higher_columns(train_data, higher_columns, higher_times)
X_train, X_test, y_train, y_test = train_test_split(data_higher.drop(['target'], axis=1), data_higher['target'], test_size=0.2, random_state=42)
# X_train, y_train = sm.fit_resample(data_higher, data_higher['target'])
# smote
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)
X_test, y_test = sm.fit_resample(X_test, y_test)

### 训练不同的模型，使用不同的特征选择方法，选择出特征后，使用不同的方法来训练模型

In [ ]:
final_results = pd.DataFrame(columns=['method', 'model', 'n', 'acc', 'auc', 'f1', 'precision', 'recall', 'confusionmatrix','coef', 'coef_rounded', 'found_coef', 'multiple_times','intercept', 'gene_id', 'gene_symbol', 'gb_acc'])
max_run_num = 300
C_values = np.logspace(-9, 0, num=max_run_num, base=10)
alpha_values = np.logspace(-5, 15, num=max_run_num, base=10)

lsvc = LinearSVC(penalty="l1", dual=False)
lsvc_params = [{'C': C} for C in C_values]
evaluate_classifier(lsvc, 'LinearSVC', 'lsvc', X_train, y_train, X_test, y_test, lsvc_params)
lr = LogisticRegression(penalty='l1', solver='liblinear')
lr_params = [{'C': C} for C in C_values]
evaluate_classifier(lr, 'LogisticRegression', 'lr', X_train, y_train, X_test, y_test, lr_params)
lasso = Lasso(alpha=0.1, max_iter=5000)
lasso_params = [{'alpha': alpha} for alpha in alpha_values]
evaluate_classifier(lasso, 'lasso', 'lasso', X_train, y_train, X_test, y_test, lasso_params)

In [ ]:
# Elastic-Net
en = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=5000)
en_params = [{'alpha': alpha, 'l1_ratio': l1_ratio} for alpha in alpha_values for l1_ratio in np.linspace(0, 1, num=5)]
evaluate_classifier(en, 'ElasticNet', 'en', X_train, y_train, X_test, y_test, en_params)

In [ ]:
# 复制 'final_results' 到 'final_results_unique' 
final_results_unique = final_results.copy()

# 将 'coef_rounded' 和 'gene_id' 列转化为字符串
final_results_unique['coef_rounded'] = final_results_unique['coef_rounded'].astype('str')
final_results_unique['gene_id'] = final_results_unique['gene_id'].astype('str')

# 删除重复行
final_results_unique.drop_duplicates(subset=['coef_rounded', 'gene_id'], keep='first', inplace=True)
final_results_unique['coef_rounded'] = final_results_unique['coef_rounded'].apply(ast.literal_eval)
final_results_unique['gene_id'] = final_results_unique['gene_id'].apply(ast.literal_eval)
final_results_unique

In [ ]:
sort_columns = ['found_coef','auc', 'recall', 'n']
sort_order = [False,False, False, True]

# Sort the DataFrame
results_sorted = final_results_unique.sort_values(by=sort_columns, ascending=sort_order).reset_index(drop=True).copy()
results_sorted_view = results_sorted.head(5).copy().drop(['coef','gene_id','intercept','multiple_times'], axis=1)
results_sorted_view

# todo: 当连续几个的gb_acc和coef_rounded都和上一个一样时，不用再显示出来。

In [ ]:
results_sorted_view

In [ ]:
# reproduce
results_sorted_found_coef = results_sorted[results_sorted['found_coef'] == True].reset_index(drop=True).copy()
reperduce = []
for i in range(len(results_sorted_found_coef)):
    gene_ids = results_sorted_found_coef['gene_id'][i]
    # check if gene_ids are in X_test columns
    if set(gene_ids).issubset(set(X_test.columns)):
        X_test_selected = X_test[gene_ids].copy()
    else:
        # modify gene_ids to match X_test columns
        gene_ids = [col for col in gene_ids if col in X_test.columns]
        X_test_selected = X_test[gene_ids].copy()
    y_test_selected = y_test
    # calculate score
    coef = results_sorted_found_coef['coef_rounded'][i]
    times = results_sorted_found_coef['multiple_times'][i]
    intercept = results_sorted_found_coef['intercept'][i]
    model_name = results_sorted_found_coef['model'][i]

    scores = []
    predictions = []
    
    # determine model and calculate scores and predictions
    if model_name == 'LinearRegression':
        for j in range(len(X_test_selected)):
            score = np.dot(X_test_selected.iloc[j], coef) / times + intercept
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
    elif model_name == 'RidgeClassifier' or model_name == 'RidgeClassifierCV':
        for j in range(len(X_test_selected)):
            score = np.dot(X_test_selected.iloc[j], coef) / times + intercept
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
    elif model_name == 'LogisticRegression' or model_name == 'LogisticRegressionCV':
        for j in range(len(X_test_selected)):
            z = np.dot(X_test_selected.iloc[j], coef) / times + intercept
            score = 1 / (1 + np.exp(-z))
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
    elif model_name == 'Lasso' or model_name == 'LassoCV':
        for j in range(len(X_test_selected)):
            score = np.dot(X_test_selected.iloc[[j]], coef) / times + intercept
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
    elif model_name == 'LinearSVC' or model_name == 'NuSVC':
        for j in range(len(X_test_selected)):
            score = np.dot(X_test_selected.iloc[j], coef) / times + intercept
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
    elif model_name == 'ElasticNet':
        for j in range(len(X_test_selected)):
            score = np.dot(X_test_selected.iloc[[j]], coef) / times + intercept
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
    elif model_name == 'SGDClassifier':
        for j in range(len(X_test_selected)):
            score = np.dot(X_test_selected.iloc[j], coef) / times + intercept
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
    else:
        # handle SVC
        for j in range(len(X_test_selected)):
            score = np.dot(X_test_selected.iloc[j], coef) / times + intercept
            prediction = 1 if score > 0.5 else 0
            scores.append(score)
            predictions.append(prediction)
        
    # calculate evaluation metrics
    acc = accuracy_score(y_test_selected, predictions)
    auc = roc_auc_score(y_test_selected, scores)
    f1 = f1_score(y_test_selected, predictions)
    precision = precision_score(y_test_selected, predictions)
    recall = recall_score(y_test_selected, predictions)

    # 能复现
    can_reproduce = True

    # if not acc == results_sorted_found_coef['acc'][i]:
    #     can_reproduce = False
    #     # print results
    #     print('Model:', model_name, 'n:', results_sorted_found_coef['n'][i],'acc:', acc, 'auc:', auc, 'f1:', f1, 'precision:', precision, 'recall:', recall)
    #     print(acc)
    #     print(results_sorted_found_coef['acc'][i])
    
    reperduce.append(can_reproduce)

results_sorted_found_coef['reproduce'] = reperduce
results_sorted_found_coef_view = results_sorted_found_coef[results_sorted_found_coef['reproduce'] == True].copy().drop(['coef','gene_id','intercept','multiple_times'], axis=1)
results_sorted_found_coef_view.head(5)
    

# 测试

### 我们的模型

In [ ]:
# 选择的模型
best_model = results_sorted_found_coef.iloc[0].copy()
weight = best_model.coef_rounded
feature =  best_model.gene_symbol
times = best_model.multiple_times
intercept = best_model.intercept
model = best_model.model

# 实际输入的模型
print('feature:', feature)
print('weight:', weight)

# up means weight > 0
# down means weight < 0
up_feature = []
up_weight = []
down_feature = []
down_weight = []
for i in range(len(weight)):
    if weight[i] > 0:
        up_feature.append(feature[i])
        up_weight.append(weight[i])
    else:
        down_feature.append(feature[i])
        down_weight.append(-1*weight[i])
up_times = times
down_times = times
up_intercept = best_model.intercept
down_intercept = best_model.intercept
up_model = best_model.intercept
down_model = best_model.intercept

In [ ]:
up_score = cal_score_with_feature(gsedata, gpl, up_feature, up_weight, up_times, up_intercept, model=up_model)
down_score = cal_score_with_feature(gsedata, gpl, down_feature, down_weight, down_times, down_intercept, model=down_model)
target = gsedata['target']
merge = pd.concat([up_score, down_score, target], axis=1)
merge.columns = ['up', 'down', 'target']
merge_by_target = merge.groupby('target')
# the dict of the target and how many times it appears
target_dict = dict(merge_by_target.size())

# where the target should be
# non-infectious illness：up和down都小于0.5
# bacterial：down大于0.5，且up<down
# viral：up大于0.5，且up>down
conditions = [
    (merge['up'] < 0.5) & (merge['down'] < 0.5),
    (merge['up'] < 0.5) & (merge['down'] > 0.5),
    (merge['up'] > 0.5) & (merge['down'] < 0.5),
    (merge['up'] > 0.5) & (merge['down'] > 0.5) & (merge['up'] > merge['down']),
    (merge['up'] > 0.5) & (merge['down'] > 0.5) & (merge['up'] < merge['down'])
]
place_for_text = [
    (0.25, 0.25),
    (0.25, 0.75),
    (0.75, 0.25),
    # (0.75, 0.75), should be righter and lower
    (0.9, 0.6),
    (0.6, 0.9)
]
target_map = [
    'non-infectious illness',
    'bacterial',
    'viral',
    'viral',
    'bacterial'
]
merge['predict'] = np.select(conditions, target_map)
merge['right'] = merge['target'] == merge['predict']
merge['area'] = np.select(conditions, ['1', '2', '3', '4', '5'])

# accuracy
accuracy_score(merge['target'], merge['predict'])

# plot
plt.figure(figsize=(10, 10))
# sns.scatterplot(data=merge, x='up', y='down', hue='target', style='right', s=100) # style='right' will make those True points drawn as x. Should fix it.
sns.scatterplot(data=merge, x='up', y='down', hue='target', style='right',style_order=[True,False], s=30, size='right', sizes=[100, 30])
plt.plot([0.5, 1], [0.5, 1], color='red', linewidth=2)
# x=0.5, y 0~0.5
plt.plot([0.5, 0.5], [0, 0.5], color='red', linewidth=2)
# x=0.5~1, y=0.5
plt.plot([0, 0.5], [0.5, 0.5], color='red', linewidth=2)
# x=0.5~1, y=0.5~1, width=0.5, 
plt.plot([0.5, 1], [0.5, 0.5], color='red', linewidth=0.5, linestyle='dotted')
plt.plot([0.5, 0.5], [0.5, 1], color='red', linewidth=0.5, linestyle='dotted')

# plot the area
merge_by_area = merge.groupby('area')
for area in ['1', '2', '3', '4', '5']:
    try:
        area_data = merge_by_area.get_group(area)
        area_data_confusion_matrix = confusion_matrix(area_data['target'], area_data['predict'])
        text = f'Area {area}\n'
        for i in range(len(area_data_confusion_matrix)):
            for j in range(len(area_data_confusion_matrix)):
                text += f'|{area_data_confusion_matrix[i][j]}'
            text += '|\n'
        plt.text(place_for_text[int(area)-1][0], place_for_text[int(area)-1][1], text, fontsize=12)
    except:
        # plt.text(place_for_text[int(area)-1][0], place_for_text[int(area)-1][1], f'Area {area}\n0\n0\n0', fontsize=12)
        pass

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


# 在右边显示各个target的数量
for i in range(len(target_dict)):
    plt.text(3.5, 0.7 - 0.08 * i, f'{list(target_dict.keys())[i]}: {list(target_dict.values())[i]}', fontsize=12)
    # 实际被预测为该target的数量
    # plt.text(1.11, 1.01 - 0.08 * i, f'Predicted: {len(merge[merge["predict"] == list(target_dict.keys())[i]])}, Accuracy: {accuracy_score(merge[merge["target"] == list(target_dict.keys())[i]]["target"], merge[merge["target"] == list(target_dict.keys())[i]]["predict"])}', fontsize=12)
    # acc only keep 2 digits and use %
    plt.text(3.5, 0.66 - 0.08 * i, f'Predicted: {len(merge[merge["predict"] == list(target_dict.keys())[i]])}, Accuracy: {round(accuracy_score(merge[merge["target"] == list(target_dict.keys())[i]]["target"], merge[merge["target"] == list(target_dict.keys())[i]]["predict"]), 2) * 100}%', fontsize=12)
# confusion matrix , also plot the text
confusion_matrix_text = confusion_matrix(merge['target'], merge['predict'])
text = 'Confusion Matrix\n'
for i in range(len(confusion_matrix_text)):
    for j in range(len(confusion_matrix_text)):
        text += f'|{confusion_matrix_text[i][j]}'
    text += '|'
    text += '\n'
plt.text(3.5, 0.3, text, fontsize=12)

# accuracy
plt.text(3.5, 0.2, f'Accuracy: {accuracy_score(merge["target"], merge["predict"])}', fontsize=12)

plt.show()

In [ ]:
test_data = train_data
test_data['target'] = test_data['target'].map(target_map)

merge_2 = train_data.copy()

viral_score = merge_2['up']
bacterial_score = merge_2['down']
target = train_data
# score is  viral_score - bacterial_score + 0.5
# viral_score,  bacterial_score and score are list.
score = np.array(viral_score) - np.array(bacterial_score) + 0.5

plot_all(
    score, 
    target,
    title="分数分布图、ROC曲线、PR曲线和混淆矩阵",
    feature=feature,
    weight=weight
)

### 文献的模型

In [ ]:
# 文献中模型
# 病毒高的
up_feature =  ['IFIT1', 'TRDV3', 'SIGLEC1','LY6E']
up_weight = [1,1 ,4, 4]
up_times = 19000
up_intercept = -0.5
up_model = 'LogisticRegression'
# 细菌高的
down_feature = ['ARG1', 'CD177', 'VNN1']
down_weight = [1, 4, 5]
down_times = 30000
down_intercept = -0.5
down_model = 'LogisticRegression'
# 实际输入的模型
feature =  up_feature + down_feature
print('feature:', feature)
weight = up_weight + (np.array(down_weight) * -1).tolist()
print('weight:', weight)

In [ ]:
up_score = cal_score_with_feature(gsedata, gpl, up_feature, up_weight, up_times, up_intercept, model=up_model)
down_score = cal_score_with_feature(gsedata, gpl, down_feature, down_weight, down_times, down_intercept, model=down_model)
target = gsedata['target']
merge = pd.concat([up_score, down_score, target], axis=1)
merge.columns = ['up', 'down', 'target']
merge_by_target = merge.groupby('target')
# the dict of the target and how many times it appears
target_dict = dict(merge_by_target.size())

# where the target should be
# non-infectious illness：up和down都小于0.5
# bacterial：down大于0.5，且up<down
# viral：up大于0.5，且up>down
conditions = [
    (merge['up'] < 0.5) & (merge['down'] < 0.5),
    (merge['up'] < 0.5) & (merge['down'] > 0.5),
    (merge['up'] > 0.5) & (merge['down'] < 0.5),
    (merge['up'] > 0.5) & (merge['down'] > 0.5) & (merge['up'] > merge['down']),
    (merge['up'] > 0.5) & (merge['down'] > 0.5) & (merge['up'] < merge['down'])
]
place_for_text = [
    (0.25, 0.25),
    (0.25, 0.75),
    (0.75, 0.25),
    # (0.75, 0.75), should be righter and lower
    (0.9, 0.6),
    (0.6, 0.9)
]
target_map = [
    'non-infectious illness',
    'bacterial',
    'viral',
    'viral',
    'bacterial'
]
merge['predict'] = np.select(conditions, target_map)
merge['right'] = merge['target'] == merge['predict']
merge['area'] = np.select(conditions, ['1', '2', '3', '4', '5'])

# accuracy
accuracy_score(merge['target'], merge['predict'])

# plot
plt.figure(figsize=(10, 10))
# sns.scatterplot(data=merge, x='up', y='down', hue='target', style='right', s=100) # style='right' will make those True points drawn as x. Should fix it.
sns.scatterplot(data=merge, x='up', y='down', hue='target', style='right',style_order=[True,False], s=30, size='right', sizes=[100, 30])
plt.plot([0.5, 1], [0.5, 1], color='red', linewidth=2)
# x=0.5, y 0~0.5
plt.plot([0.5, 0.5], [0, 0.5], color='red', linewidth=2)
# x=0.5~1, y=0.5
plt.plot([0, 0.5], [0.5, 0.5], color='red', linewidth=2)
# x=0.5~1, y=0.5~1, width=0.5, 
plt.plot([0.5, 1], [0.5, 0.5], color='red', linewidth=0.5, linestyle='dotted')
plt.plot([0.5, 0.5], [0.5, 1], color='red', linewidth=0.5, linestyle='dotted')

# plot the area
merge_by_area = merge.groupby('area')
for area in ['1', '2', '3', '4', '5']:
    try:
        area_data = merge_by_area.get_group(area)
        area_data_confusion_matrix = confusion_matrix(area_data['target'], area_data['predict'])
        text = f'Area {area}\n'
        for i in range(len(area_data_confusion_matrix)):
            for j in range(len(area_data_confusion_matrix)):
                text += f'|{area_data_confusion_matrix[i][j]}'
            text += '|\n'
        plt.text(place_for_text[int(area)-1][0], place_for_text[int(area)-1][1], text, fontsize=12)
    except:
        # plt.text(place_for_text[int(area)-1][0], place_for_text[int(area)-1][1], f'Area {area}\n0\n0\n0', fontsize=12)
        pass

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


# 在右边显示各个target的数量
for i in range(len(target_dict)):
    plt.text(1.1, 0.7 - 0.08 * i, f'{list(target_dict.keys())[i]}: {list(target_dict.values())[i]}', fontsize=12)
    # 实际被预测为该target的数量
    # plt.text(1.11, 1.01 - 0.08 * i, f'Predicted: {len(merge[merge["predict"] == list(target_dict.keys())[i]])}, Accuracy: {accuracy_score(merge[merge["target"] == list(target_dict.keys())[i]]["target"], merge[merge["target"] == list(target_dict.keys())[i]]["predict"])}', fontsize=12)
    # acc only keep 2 digits and use %
    plt.text(1.1, 0.66 - 0.08 * i, f'Predicted: {len(merge[merge["predict"] == list(target_dict.keys())[i]])}, Accuracy: {round(accuracy_score(merge[merge["target"] == list(target_dict.keys())[i]]["target"], merge[merge["target"] == list(target_dict.keys())[i]]["predict"]), 2) * 100}%', fontsize=12)
# confusion matrix , also plot the text
confusion_matrix_text = confusion_matrix(merge['target'], merge['predict'])
text = 'Confusion Matrix\n'
for i in range(len(confusion_matrix_text)):
    for j in range(len(confusion_matrix_text)):
        text += f'|{confusion_matrix_text[i][j]}'
    text += '|'
    text += '\n'
plt.text(1.1, 0.3, text, fontsize=12)

# accuracy
plt.text(1.1, 0.2, f'Accuracy: {accuracy_score(merge["target"], merge["predict"])}', fontsize=12)

plt.show()

In [ ]:
test_data = gsedata[gsedata['target'].isin(['viral', 'bacterial'])]
test_data['target'] = test_data['target'].map({'viral': 1, 'bacterial': 0})

merge_2 = merge[merge['target'] != 'non-infectious illness'].copy()

viral_score = merge_2['up']
bacterial_score = merge_2['down']
target = merge_2['target'].map({'viral': 1, 'bacterial': 0})

# score is  viral_score - bacterial_score + 0.5
# viral_score,  bacterial_score and score are list.
score = np.array(viral_score) - np.array(bacterial_score) + 0.5

plot_all(
    score, 
    target,
    title="分数分布图、ROC曲线、PR曲线和混淆矩阵",
    feature=feature,
    weight=weight
)